# Initializing

In [1]:
from scipy.signal import convolve2d
from matplotlib import pyplot as plt
import numpy as np
import cv2
from PIL import Image
from numpy import asarray
from pyperclip import copy
from pyzbar.pyzbar import decode
import urllib.request
import requests
%matplotlib inline

In [7]:
def BarcodeReader(image):
    img = cv2.imread(image)
    detectedBarcodes = decode(img)
    if not detectedBarcodes:
        print("Barcode Not Detected or your barcode is blank/corrupted!")
    else:
        for barcode in detectedBarcodes: 
            (x, y, w, h) = barcode.rect
            cv2.rectangle(img, (x-10, y-10),
                          (x + w+10, y + h+10),
                          (255, 0, 0), 2)
            if barcode.data!="":
                return str(int(barcode.data))
                 
def ISBNcover(isbn):
    return'https://d1w7fb2mkkr3kw.cloudfront.net/assets/images/book/lrg/' + str(int(isbn))[:4] + '/' + str(int(isbn))[4:8] + '/' + str(int(isbn)) + '.jpg'

def save(weblink):
    urllib.request.urlretrieve(weblink, "coverphoto.jpg")

def pricegenerator(isbn):
    page = requests.get('https://www.bookdepository.com/search/' + str(isbn))
    htmlcode1 = str(page.text)
    start = htmlcode1.find('<p class="list-price">')
    htmlcode2 = htmlcode1[start+22:].strip()[15:].strip()
    end = htmlcode2.find('</p>')
    htmlcodesnippet = htmlcode2[:end]
    return htmlcodesnippet

def lineg(filename):
    original_img = Image.open(filename + '.jpg').convert('L')
    original_img = np.array(original_img)
    original_img = original_img/255
    fil = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
    fmap2 = convolve2d(original_img,fil,mode='same')
    plt.axis('off')
    plt.imsave(filename + 'line' + '.jpg', fmap2, cmap='gray')
    plt.close()
    image_path = filename + 'line' + '.jpg'
    image_file = Image.open(image_path)
    image_file.save(filename + 'line' + '.jpg', quality=95)
def testimg(n1='coverphoto', n2='coverphotoused'):
    img1 = Image.open(n1 + 'line.jpg')
    image1 = asarray(img1)
    img2 = Image.open(n2 + 'line.jpg')
    image2 = asarray(img2)
    diff = np.absolute(np.subtract(image1, image2))
    imgdiff = Image.fromarray(diff)
    imgdiff.save('diff.jpg')
    return round(np.average(diff))/6.25+17.5
def checkimg(coverphoto = 'coverphoto', nia='nia.jpg'):
    coverimg = Image.open(coverphoto + '.jpg')
    niaimg = Image.open(nia)
    cover = asarray(coverimg)
    noimageava = asarray(niaimg)
    if cover.all() == noimageava.all():
        return False
    return True

def pricetag(pc,price):
    override = input(f'Automatic price generated: {price} \n Manual Override T/F ')
    if override.lower() == 't':
        price = input('New price: ')
    else:
        price = str(price * pc)
    copy(price)
    return price

In [9]:
def full(used = 'coverphotoused', barcode = 'bb'):
    save(ISBNcover(BarcodeReader('bb.jpg')))
    price = pricegenerator(BarcodeReader('bb.jpg'))
    if not checkimg():
        return 'Not avaliable'
    lineg('coverphoto')
    lineg('coverphotoused')
    pc = testimg()
    pricetag(pc, price)


In [12]:
full()

ValueError: operands could not be broadcast together with shapes (400,267,3) (718,479,3) 